[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Denis-R-V/Homeworks/blob/main/Task_1.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"

In [4]:
# Импорт библиотек
import requests
import pandas as pd

## Список объектов

In [6]:
# Для скачивания данных по блоками по 100
# Задание параметров: начальное смещение равно 1, количество записей в блоке равно 100
offset_ = 10001
limit_ = 100
# Пустой список ID объектов
objids = []
# Цикл из 1 млн итераций по 100 объектов (до 100млн домой, на сайте их 10582)
for i in range(1, 1000000):
# Выполняем запрос к сайте, выдергиваем json
    try:
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
        res = requests.get(url)
        objects_data = res.json()
# Проверяем, есть ли в ответе объект
        objects_data.get('data').get('list')[0]
# Объекта нет - прерываем цикл
    except IndexError:
#        print('прерывание')
        break
# Не возникло исключение - берем ID объектов и добавляем их в список objids
    else:
        objects_list = objects_data.get('data').get('list')
        objids.extend([x.get('objId') for x in objects_list])
# увеличиваем смещение на 100
        offset_ += 100
len(objids)

582

# Данные по определенному объекту

In [ ]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/29679'

In [ ]:
res = requests.get(url)
res.json()